In [93]:
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

In [94]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [139]:
my_page=simple_get('https://www.instagram.com/p/CDYa6rGFXtB/')

In [140]:
len(my_page)

88848

In [141]:
my_page_html = BeautifulSoup(my_page)

In [98]:
import re
pattern = re.compile(r'\"text\":')

In [142]:
for script in my_page_html.find_all('script'):
    if "PostPage" in script.text:
        s=script.text.rstrip(';')
        s=s.lstrip('window._sharedData = ')
        print(s)
        break

{"config":{"csrf_token":"Wz8XX0afpO8M1gKGrihNflJyt2ydnWU1","viewer":null,"viewerId":null},"country_code":"ID","language_code":"en","locale":"en_US","entry_data":{"PostPage":[{"graphql":{"shortcode_media":{"__typename":"GraphImage","id":"2366759985395104577","shortcode":"CDYa6rGFXtB","dimensions":{"height":1208,"width":1080},"gating_info":null,"fact_check_overall_rating":null,"fact_check_information":null,"sensitivity_friction_info":null,"media_overlay_info":null,"media_preview":"ACYq0p7qNM4LA5OeWx745/z6VD9uQ4ZSxzzgFv5Z+ucZqjcGMMUHz7iTn0zn+X/1qiZIo0DHCyDtjr7jHb+vFIdjQbUo167/AMd3+P1qxHcLLjaxG4ZGSw749axZZRneNrDjjOSM5Jz689M9KFmAhMrHBDbQuOuQT1zx657ccHNGo3Y3/tUcbY+ckj3I4x79f/r0Vh20zI5jbcrY3Hn1xxj8c0UxFeVjubHXLD/x41VnwQpB55yO2R/jWnfw7ZVKj5ZG2n2Oe31HP1plzapFGzEMxAwD6e5/rRfYLbmZG+1GH94rg/TJP9Ks+YghUEYBlBYZzwFGT7Zz0pskeIOO8uB/3zj+dWEjC6ezHqzbh+YAp3FYe9wlzdNJH90p3GORgUVNo0CHexAbnAJ9B1/pRSuFizdQNK6EchHyR/X8KnkiEilW6HrirpUZPApNo9BWZoZFzZ+YFVcKobcT+GPxJ9aebYGHyAwxgAn2zn861NinqAfwpfKT+6PyFAipaQrDlUPH

In [152]:
import json
s_json=json.loads(s)
entry_data = s_json['entry_data']
#print(data)

In [155]:
import pandas as pd
parent_comments=[]
edge_comments=[]
p_username=[]
e_username=[]
for page in entry_data['PostPage']:
    count = page['graphql']['shortcode_media']['edge_media_to_parent_comment']['count']
    print(count)
    comment_list = page['graphql']['shortcode_media']['edge_media_to_parent_comment']['edges']
    for comment in comment_list:
        #print(comment['node']['text']+" by " + comment['node']['owner']['username'])
        parent_comments.append(comment['node']['text'])
        p_username.append(comment['node']['owner']['username'])
        edge_comment_list = comment['node']['edge_threaded_comments']['edges']
        for edge_comment in edge_comment_list:
            #print("Edge Comment: " + edge_comment['node']['text'] + " by " + edge_comment['node']['owner']['username'])
            edge_comments.append(edge_comment['node']['text'])
            e_username = edge_comment['node']['owner']['username']

181


In [157]:
    data = {'username': p_username, 'comment' : parent_comments}
    df=pd.DataFrame(data)
    df2 = pd.DataFrame({'username': e_username, 'comment' : edge_comments})
    df.append(df2)
    len(df)
    df

,username,comment
0,prashantpitambar,🤣🤣🤣🤣🤣 True
1,the.kannan,🤣🤣🤣🤣
2,ameen___p,"""the PR goes brrrraa"""
3,geokosh,Daaamn i see it now😂😂🔥🔥👌
4,thesitcomenthusiast,Double🔥
5,kvin4real,Commies With Attitude🔥
6,_yadu_krish__,'Golden'❤
7,cl0udy_b0y,Boy N Hood
8,bharath_manunited,😂😂
9,amel_joseph,GOLD RAJA FT. PINARAYI 📀
